In [1]:
#%pip install git+https://github.com/cair/tmu.git
#%pip install numpy==1.26.4

In [1]:
import numpy as np
import pickle
import os

from tmu.models.autoencoder.autoencoder import TMAutoEncoder

2025-03-18 10:57:11,784 - tmu.util.cuda_profiler - WARNING - Could not import pycuda: No module named 'pycuda'
2025-03-18 10:57:11,786 - tmu.clause_bank.clause_bank_cuda - ERROR - No module named 'pycuda'
Traceback (most recent call last):
  File "/Users/kjellhaaland/Documents/GitHub/uia-master-thesis/.venv/lib/python3.12/site-packages/tmu/clause_bank/clause_bank_cuda.py", line 41, in <module>
    from pycuda._driver import Device, Context
ModuleNotFoundError: No module named 'pycuda'
2025-03-18 10:57:11,796 - tmu.clause_bank.clause_bank_cuda - WARNING - Could not import pycuda. This indicates that it is not installed! A possible fix is to run 'pip install pycuda'. Fallback to CPU ClauseBanks.


In [2]:
# Create folder models if it does not exist
os.makedirs("models", exist_ok=True)

In [3]:
def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X

In [4]:
def save_model(tm: TMAutoEncoder, filename: str):
    a, d = tm.X_train, tm.encoded_X_train

    tm.X_train = None
    tm.encoded_X_train = None

    with open(f"./models/{filename}", "wb") as f:
        pickle.dump(tm, f)

    tm.X_train = a
    tm.encoded_X_train = d

In [5]:
# TODO: Load X_train from a dataset

train_datasets = [83, 52, 21, 2, 23, 87, 74, 86, 82]

# Load all datasets into one array
X_train = np.concatenate([load_train_dataset("B", i) for i in train_datasets])

print(f"X_train shape: {X_train.shape}")

X_train shape: (418792, 315)


In [6]:
def train(args):
    tm = TMAutoEncoder(
        number_of_clauses=args["num_clauses"],
        T=args["T"],
        s=args["s"],
        output_active=args["output_active"],
        max_included_literals=args["max_included_literals"],
        accumulation=args["accumulation"],
        feature_negation=args["feature_negation"],
        platform=args["platform"],
        output_balancing=args["output_balancing"],
    )

    print(f"Starting training for {args['epochs']} epochs")

    for e in range(args["epochs"]):
        tm.fit(X_train, number_of_examples=args["number_of_examples"])

        save_model(tm, f"latest_{e}.pkl")
        print(f"Epoch: {e + 1}")


In [7]:
number_of_features = X_train.shape[1]
print(number_of_features)

315


In [8]:
number_of_features = X_train.shape[1]
output_active = np.arange(number_of_features, dtype=np.uint32)

number_of_clauses = 500

args: dict = {
    "clause_weight_threshold": 1,
    "number_of_examples": 40000,
    "output_active": output_active,
    "accumulation": 1,
    "num_clauses": number_of_clauses,
    "T": int(number_of_clauses * 0.75 * 100),
    "s": 25,
    "epochs": 25,
    "platform": "CPU",
    "output_balancing": 0,
    "max_included_literals": number_of_features,
    "feature_negation": True,
}

result = train(args)

Starting training for 25 epochs


KeyboardInterrupt: 